In [1]:
#!nvidia-smi

In [ ]:
#!pip install opencv-python numpy torch torchvision ultralytics matplotlib tqdm timm
#!pip install -q transformers sentencepiece gdown inference-gpu supervision more_itertools scikit-learn umap-learn
#!pip install "inference[transformers,sam,clip,gaze,grounding-dino]"

: 

In [1]:
from processingVideo.utils import read_video, save_video
from processingVideo.tracker import Tracker
from processingVideo.team_assigner import TeamAssigner
from processingVideo.pitch import PitchAnnotator, SoccerPitchConfiguration

import torch
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

CONFIG = SoccerPitchConfiguration()

cuda


In [6]:
# 1. load all frames
video_frames = read_video('media/inputs/08fd33_4.mp4')

# 2. track players/ref/ball in the original video
tracker = Tracker('processingVideo/models/player_detection.pt')
tracks  = tracker.get_object_tracks(video_frames)
tracker.add_position_to_track(tracks)

# 3. assign each tracked player a team ID/color
team_assigner = TeamAssigner(device='cuda')
team_assigner.assign_teams(tracks, video_frames)


0: 576x1024 1 ball, 1 goalkeeper, 21 players, 3 referees, 35.3ms
1: 576x1024 1 ball, 1 goalkeeper, 21 players, 3 referees, 35.3ms
2: 576x1024 1 ball, 1 goalkeeper, 21 players, 3 referees, 35.3ms
3: 576x1024 1 ball, 1 goalkeeper, 23 players, 3 referees, 35.3ms
4: 576x1024 1 ball, 1 goalkeeper, 23 players, 3 referees, 35.3ms
5: 576x1024 1 ball, 1 goalkeeper, 23 players, 3 referees, 35.3ms
6: 576x1024 1 ball, 1 goalkeeper, 27 players, 3 referees, 35.3ms
7: 576x1024 1 ball, 1 goalkeeper, 21 players, 3 referees, 35.3ms
8: 576x1024 1 ball, 25 players, 3 referees, 35.3ms
9: 576x1024 1 ball, 21 players, 3 referees, 35.3ms
10: 576x1024 1 ball, 1 goalkeeper, 24 players, 3 referees, 35.3ms
11: 576x1024 1 goalkeeper, 24 players, 3 referees, 35.3ms
12: 576x1024 1 ball, 1 goalkeeper, 27 players, 3 referees, 35.3ms
13: 576x1024 1 ball, 1 goalkeeper, 24 players, 3 referees, 35.3ms
14: 576x1024 1 ball, 25 players, 3 referees, 35.3ms
15: 576x1024 1 ball, 21 players, 3 referees, 35.3ms
16: 576x1024 1 ba

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Embedding extraction: 17it [00:02,  8.30it/s]
Embedding extraction: 480it [00:58,  8.16it/s]


In [7]:
# Video with players detected and teams drawn
#-----------------

output_video_frames = tracker.draw_annotations(video_frames, tracks) #team_ball_control

save_video(output_video_frames, 'output_video.avi')

#-----------------

In [9]:
# 4. annotate the pitch

pitch_ann = PitchAnnotator(
    CONFIG=CONFIG,
    model_path = "processingVideo/models/field_detection.pt"
)

In [10]:
results = pitch_ann.annotate_video_batched(
    video_frames=video_frames,
    batch_size=8,   # try 8–16 
)

In [6]:
# Pitch Detection, adds edges and vertices drawn
#---------------

output_frames = []
for i, (frame, res) in enumerate(zip(video_frames, results)):
    fr = pitch_ann.annotate_frame_from_result(frame, res)
    output_frames.append(fr)

save_video(output_frames, 'pitch_with_edges.avi')

#---------------

In [7]:
# Tactical board pitch conversion
#-----------------

tactical_board = []
for i, (frame, res) in enumerate(zip(video_frames, results)):
    tb_frame = pitch_ann.annotate_tactical_board_from_result(frame, tracks, i, CONFIG, res, kp_thresh=0.5)
    tactical_board.append(tb_frame)

save_video(tactical_board, 'match_with_tactical_board.avi')

#-----------------

KeyError: 'position'

In [ ]:
# Voronoi pitch detection
#-----------------

voronoi_board = []
for i, (frame, res) in enumerate(zip(video_frames, results)):
    vb_frame = pitch_ann.annotate_voronoi_from_result(frame, tracks, i, CONFIG, res, kp_thresh=0.5, vor_step=3)
    voronoi_board.append(vb_frame)

save_video(voronoi_board, 'match_with_voronoi.avi')

#-----------------

KeyError: 'position'

In [11]:
pd_frames, tb_frames, vb_frames = [], [], []
for idx, (frame, res) in enumerate(zip(video_frames, results)):
    pd, tb, vb = pitch_ann.annotate_all_from_result(frame, tracks, idx, CONFIG, result=res, kp_thresh=0.5)
    pd_frames.append(pd)
    tb_frames.append(tb)
    vb_frames.append(vb)


KeyError: 'position'

In [7]:
save_video(pd_frames, 'pitch_with_edges.avi')
save_video(tb_frames, 'match_with_tactical_board.avi')
save_video(vb_frames, 'match_with_voronoi.avi')